<a href="https://colab.research.google.com/github/neverlast121/Feedforward_NeuralNetwork/blob/main/Feed_forward_NN_MNIST_Pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
input_size = 784 # 28*28
hidden_size = 100
num_classes = 10
num_epoch = 1
batch_size = 100
learning_rate = 1e-3

In [ ]:
mnist_train = datasets.MNIST(
    root = 'data',
    download = True,
    train = True,
    transform = ToTensor()
)

mnist_test = datasets.MNIST(
    root = 'data',
    download = True,
    train = False,
    transform = ToTensor()
)

In [ ]:
train_loader = DataLoader(
    dataset = mnist_train,
    batch_size = batch_size,
    shuffle = True
)

test_loader = DataLoader(
    dataset = mnist_test,
    batch_size = batch_size,
    shuffle = False
)

In [ ]:
class FeedFNN(nn.Module):
  def __init__(self, input_size, hidden_size, num_classes):
    super().__init__()
    self.FeedNN = nn.Sequential(
        nn.Linear(input_size, hidden_size),
        nn.ReLU(),
        nn.Linear(hidden_size, num_classes)
    )
  def forward(self, x):
    out = self.FeedNN(x)
    return out

model = FeedFNN(input_size, hidden_size, num_classes)

In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

In [ ]:
n_total_step = len(train_loader)
for epoch in range(num_epoch):
  for i, (images, labels) in enumerate(train_loader):
    images = images.reshape(-1, 28*28).to(device)
    labels = labels.to(device)

    # forwads
    outputs = model(images)
    loss = loss_fn(outputs, labels)

    # backwards
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (i+1) % 100 == 0:
      print(f'Epoch: {epoch+1}/ {num_epoch}, step {i+1}/{n_total_step}, loss = {loss.item():.4f}')

Epoch: 1/ 1, step 100/600, loss = 0.0091
Epoch: 1/ 1, step 200/600, loss = 0.0192
Epoch: 1/ 1, step 300/600, loss = 0.0178
Epoch: 1/ 1, step 400/600, loss = 0.0655
Epoch: 1/ 1, step 500/600, loss = 0.0128
Epoch: 1/ 1, step 600/600, loss = 0.0293


In [ ]:
with torch.no_grad():
  n_samples = 0
  n_correct = 0

  for images, labels in test_loader:
    images = images.reshape(-1, 28*28).to(device)
    labels = labels.to(device)

    outputs = model(images)

    _, predictions = torch.max(outputs, 1)
    n_samples += labels.shape[0]
    n_correct += (predictions == labels).sum().item()

  acc = 100.0 * n_correct / n_samples
  print(f'Accuracy: {acc}')

Accuracy: 97.39
